Language Modeling & Recurrent Neural Networks


In [0]:
%tensorflow_version 2.x

In [0]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=1eabf65fb4f2e64fe4f97d43ec38d5238ee965433e0df378c346026071adf645
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [0]:
import os
os.getcwd()

'/content'

time: 3.9 ms


In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks")

time: 1.07 ms


In [0]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks'

time: 2 ms


In [0]:
import tensorflow as tf
import numpy as np

time: 1.16 s


In [0]:
tf.keras.utils.get_file("/content/drive/My Drive/Colab Notebooks/shakespeare.txt", 
                        "https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt")

'/content/drive/My Drive/Colab Notebooks/shakespeare.txt'

time: 7.48 ms


In [0]:
!python prepare_data.py shakespeare.txt skp

2020-05-26 08:17:01.624566: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 22981 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900

In [0]:
from prepare_data import parse_seq
import pickle
# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
seq_len = 200
data = data.map(lambda x: parse_seq(x, seq_len))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb")) 
#unpickling-convert byte stream to python object structure #read bytes
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab_size) #26(uppercase)+26(lowercase)+16(special characters)=68
print(vocab) 
print(ind_to_ch)

68
{'z': 1, ';': 2, 'p': 3, 'M': 4, 'a': 5, 'k': 6, 'y': 7, 'l': 8, 'u': 9, 'e': 10, 'q': 11, '3': 12, 'C': 13, 'o': 14, 'R': 15, 'Q': 16, 'N': 17, 'X': 18, 'd': 19, 'P': 20, '?': 21, 'Y': 22, 'i': 23, '\n': 24, 'A': 25, 'v': 26, 's': 27, '-': 28, 'm': 29, 'I': 30, 'B': 31, ']': 32, 'U': 33, 'h': 34, 'j': 35, 'S': 36, 'F': 37, ',': 38, '!': 39, 'J': 40, 'T': 41, 'E': 42, 'n': 43, 'G': 44, 'r': 45, ':': 46, 'Z': 47, '[': 48, 'V': 49, 'w': 50, "'": 51, 'W': 52, ' ': 53, 'x': 54, 't': 55, 'D': 56, 'b': 57, '&': 58, 'c': 59, '$': 60, 'g': 61, 'O': 62, 'L': 63, 'K': 64, 'f': 65, 'H': 66, '.': 67, '<S>': 0}
{1: 'z', 2: ';', 3: 'p', 4: 'M', 5: 'a', 6: 'k', 7: 'y', 8: 'l', 9: 'u', 10: 'e', 11: 'q', 12: '3', 13: 'C', 14: 'o', 15: 'R', 16: 'Q', 17: 'N', 18: 'X', 19: 'd', 20: 'P', 21: '?', 22: 'Y', 23: 'i', 24: '\n', 25: 'A', 26: 'v', 27: 's', 28: '-', 29: 'm', 30: 'I', 31: 'B', 32: ']', 33: 'U', 34: 'h', 35: 'j', 36: 'S', 37: 'F', 38: ',', 39: '!', 40: 'J', 41: 'T', 42: 'E', 43: 'n', 44: 'G', 45

In [0]:
batch_size = 128
data = data.shuffle(10000).batch(batch_size).repeat()

time: 7.46 ms


In [0]:
# view data
print(data)
for index ,value in enumerate(data): 
  print(index) 
  print(value.shape)
  print(value[0])
  vector=tf.one_hot(value,vocab_size)  
  s=[]
  sentence=""
  for i in value[0].numpy():     
    s.append(ind_to_ch[i])
  print("----example sentence-----")
  print(sentence.join(s))
  print("----vector representation-----")
  print(vector)
  break

<RepeatDataset shapes: (None, 200), types: tf.int32>
0
(128, 200)
tf.Tensor(
[ 0 55 53 34 10 53 34  5 55 34 53 43 14 55 67 24 24 13 42 63 30 25 46 24
 17 14 38 53 34  5 55 34 53 43 14 55 21 53 15 14 27  5  8 23 43 19 53  8
  5 59  6 27 53 55 34 10 43 53 55 34 10 53  8 14 26 10 24 52 34 23 59 34
 53 55 10  5 59 34 10 55 34 53 55 34 10 10 53 55 34  5 55 53 55 34 14  9
 53  5 43 19 53 30 53  5 29 53 14 43 10 46 24 36 34  5  8  8 53 50 10 53
 57 10 53 27  9 43 19 10 45 51 19 21 53 27 34  5  8  8 53 50 10 53  3  5
 45 55 38 53 27 50 10 10 55 53 61 23 45  8 21 24 17 14 46 53  8 10 55 53
 29  7 53 65  5 55 34 10 45 53 27 10 10  6 53  5 43 14 55 34 10 45 53 34
 10 23 45 67 24 41 34 10], shape=(200,), dtype=int32)
----example sentence-----
<S>t he hath not.

CELIA:
No, hath not? Rosalind lacks then the love
Which teacheth thee that thou and I am one:
Shall we be sunder'd? shall we part, sweet girl?
No: let my father seek another heir.
The
----vector representation-----
tf.Tensor(
[[[1. 0. 0. ..

In [0]:
#set up variables
hidden_units = 512

#initializer = tf.random_uniform_initializer
initlializer = tf.keras.initializers.GlorotUniform() #Xavier uniform initializer

W_xh = tf.Variable(initlializer([vocab_size, hidden_units])) #input weights (68, 512)
W_hh = tf.Variable(initlializer([hidden_units, hidden_units]))#hidden weights (512, 512)
b_h = tf.Variable(tf.zeros([hidden_units])) #hidden bias (512)
W_ho = tf.Variable(initlializer([hidden_units, vocab_size]))#output weights (512, 68)
b_o = tf.Variable(tf.zeros([vocab_size])) #output bias (68)

varis = [W_xh, W_hh,b_h, W_ho, b_o]

time: 36.2 ms


In [0]:
train_steps = 5000
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

@tf.function
def training(input):
  with tf.GradientTape() as tape:
    hidden_state = tf.zeros([tf.shape(input)[0], hidden_units]) #(128,512)
    #sum_loss = 0
    ta_loss = tf.TensorArray('float32', size = tf.shape(input)[1] - 1)  
    #supports gradient back-propagation via special "flow" control flow dependencies #pre allocated size

    for time_step in tf.range(tf.shape(input)[1] - 1): #length is 199 after removing <S>
      #state = init_state
      input_one_hot = tf.one_hot(input[:, time_step], depth = vocab_size) #(128,200,68)
      #input is turned from batch_size x seq_len into batch_size x seq_len x vocab_size
      hidden_state = tf.nn.tanh(tf.matmul(input_one_hot, W_xh) + tf.matmul(hidden_state, W_hh) + b_h)
      logits = tf.matmul(hidden_state, W_ho) + b_o

      loss = loss_fn(input[:, time_step + 1], logits) #the targets are just the input shifted by one time step
      ta_loss = ta_loss.write(time_step, loss)
    ta_loss = ta_loss.stack() #Return the values in the TensorArray as a stacked Tensor
    avg_loss = tf.reduce_mean(ta_loss)
      #sum_loss += loss

  grads = tape.gradient(avg_loss, varis) 

  #avoid exploding gradients with gradient clipping
  grads, _ = tf.clip_by_global_norm(grads, 0.3) #clip_value = 0.3
  optimizer.apply_gradients(zip(grads, varis))
      
  return avg_loss

for step, batch in enumerate(data):
    if step > train_steps:
      break

    loss_avg = training(batch) #batch (128,200)

    if not step % 1000:
      print('Step {} Average Loss {}'.format(step, loss_avg))

Step 0 Average Loss 4.240265846252441
Step 1000 Average Loss 1.8317983150482178
Step 2000 Average Loss 1.5790228843688965
Step 3000 Average Loss 1.4850237369537354
Step 4000 Average Loss 1.419765591621399
Step 5000 Average Loss 1.3816020488739014
time: 17min 19s


In [0]:
#sample and generate new text

def generate_text(num_generate):
  hidden_state = tf.zeros([1, hidden_units]) #(1, 512)
  sample = [0]

  for i in range(num_generate):
    hidden_state = tf.nn.tanh(tf.matmul(tf.one_hot(sample[-1:], depth = vocab_size), W_xh) + tf.matmul(hidden_state, W_hh) + b_h)
    probabilities = tf.nn.softmax(tf.matmul(hidden_state , W_ho) + b_o).numpy()[0] #outputs probability distributions (68,)
    sample.append(np.random.choice(vocab_size, p = probabilities))

  output = "".join(ind_to_ch[j] for j in sample) 
  return output

num_generate = 3000
output_text = generate_text(num_generate)
print(output_text)

<S>ed did I wan without by it.
Times Pleace marry! Have I lay up verngness must
neighbour you herced! Sir Anjest falls, and readors,
Not now got one, and flowing too, heavens,
the gods of him; we know not whining of voound
Well as well for born.

SPEED:
Now, I' brulatily is not lend for him.

ANTIPHOLUS OF WIRCK:
Somp lion is voow'st.
'doughtuse and her formanged to some muster
As there we hours and prunt of thines.

KING RICHARD III:
Nay, now, A harling, like a quarrel yor
Arjouthed, indeed on him a' wit ignorant fell,
If death More lament, but well be loved you, be
quesning of me? I am her his: I will
neighbourded morke to my cotions,
I'st have it be not to lay with copress'd,
And grieved the persone. I ambark, to did;
With your bestracurs missters, and judgment; yet shall insoctity.
Humon or to one emence, and send ip wornun.

BRATON:
Where by here! Kenc'l! I proless king, hast too hour martile:
There is a palace! I would do you well.

ADRIANA:
God make a Melan, but discoury my lies